# Preprocessing


In [2]:
!pip install keras_tuner
!pip install tensorflow

import numpy as np
import os
import sys
import keras_tuner as kt
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model


In [3]:
import numpy as np
import os
import sys
from google.colab import userdata
s = userdata.get('ghkey')
with open('./github_token', 'wb') as f:
  f.write(bytes(s, 'utf'))
!rm -rf /content/207_final
!git clone https://oauth2:$(cat ./github_token)@github.com/numbersrcool/207_final.git

Cloning into '207_final'...
remote: Enumerating objects: 355, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 355 (delta 102), reused 133 (delta 72), pack-reused 179 (from 1)
Receiving objects: 100% (355/355), 147.53 MiB | 15.92 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [4]:
# Kaggle Import
import kagglehub

#from google.colab import userdata
#userdata.get('KAGGLE_USERNAME')
#userdata.get('KAGGLE_KEY')

# Download latest version
sign_language_data_path = kagglehub.dataset_download("harshvardhan21/sign-language-detection-using-images")
sign_language_data_path += "/data"
print(sign_language_data_path)

!mv /root/.cache/kagglehub/datasets/harshvardhan21/sign-language-detection-using-images/versions/1/data /content


/root/.cache/kagglehub/datasets/harshvardhan21/sign-language-detection-using-images/versions/1/data
mv: cannot stat '/root/.cache/kagglehub/datasets/harshvardhan21/sign-language-detection-using-images/versions/1/data': No such file or directory


In [5]:
%cd /content/207_final/
!git pull

/content/207_final
Already up to date.


In [6]:
os.makedirs("/content/data", exist_ok=True)
%env MIDS_207_DATA_PATH=/content/data


env: MIDS_207_DATA_PATH=/content/data


In [7]:
!ls /content/data


1  3  5  7  9  B  D  F	H  J  L  N  P  R  T  V	X  Z
2  4  6  8  A  C  E  G	I  K  M  O  Q  S  U  W	Y


In [8]:
from datasets import training


loading data from /content/data...
splitting data into train, val, test...


In [9]:
from util import imgs as imgutils


In [10]:
for i in range(5):
    x, y = training.data[i]
    print(f'batch {i} has {x.shape[0]} images')

batch 0 has 100 images
batch 1 has 100 images
batch 2 has 100 images
batch 3 has 100 images
batch 4 has 100 images


In [13]:
# Create a mapping for the provided classes
all_classes = ['1', '3', '5', '7', '9', 'B', 'D', 'F', 'H', 'J', 'L', 'N', 'P', 'R', 'T', 'V', 'X', 'Z',
               '2', '4', '6', '8', 'A', 'C', 'E', 'G', 'I', 'K', 'M', 'O', 'Q', 'S', 'U', 'W', 'Y']
label_mapping = {label: idx for idx, label in enumerate(all_classes)}
print("Label mapping:", label_mapping)


Label mapping: {'1': 0, '3': 1, '5': 2, '7': 3, '9': 4, 'B': 5, 'D': 6, 'F': 7, 'H': 8, 'J': 9, 'L': 10, 'N': 11, 'P': 12, 'R': 13, 'T': 14, 'V': 15, 'X': 16, 'Z': 17, '2': 18, '4': 19, '6': 20, '8': 21, 'A': 22, 'C': 23, 'E': 24, 'G': 25, 'I': 26, 'K': 27, 'M': 28, 'O': 29, 'Q': 30, 'S': 31, 'U': 32, 'W': 33, 'Y': 34}


# Training Data

In [14]:
all_basic_images, all_basic_labels = [], []

for i in range(50):  # Adjust the range based on the number of batches
    x, y = training.baseline_data[i]
    all_basic_images.append(x)
    all_basic_labels.append(y)

# Combine all batches into a single dataset
x_basic_train = tf.concat(all_basic_images, axis=0)
y_basic_train = np.concatenate(all_basic_labels, axis=0)

# Check the dataset shapes
print(f"x_train shape: {x_basic_train.shape}")  # Example: (num_samples, height, width, channels)
print(f"y_train shape: {y_basic_train.shape}")  # Example: (num_samples,)
print(f"Unique labels in y_train: {np.unique(y_basic_train)}")  # Verify labels are mapped correctly


x_train shape: (5000, 128, 128, 3)
y_train shape: (5000,)
Unique labels in y_train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]


In [15]:
all_augmented_images, all_augmented_labels = [], []

for i in range(50):  # Adjust the range based on the number of batches
    x, y = training.data[i]
    all_augmented_images.append(x)
    all_augmented_labels.append(y)

# Combine all batches into a single dataset
x_augmented_train = tf.concat(all_augmented_images, axis=0)
y_augmented_train = np.concatenate(all_augmented_labels, axis=0)

# Check the dataset shapes
print(f"x_train shape: {x_augmented_train.shape}")  # Example: (num_samples, height, width, channels)
print(f"y_train shape: {y_augmented_train.shape}")  # Example: (num_samples,)
print(f"Unique labels in y_train: {np.unique(y_augmented_train)}")  # Verify labels are mapped correctly


x_train shape: (5000, 128, 128, 3)
y_train shape: (5000,)
Unique labels in y_train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]


# Model Builder (4 Conv Layer)

In [16]:
from tensorflow.keras import layers, models, regularizers

num_classes = len(all_classes)  # 36 classes

import keras_tuner as kt
from tensorflow.keras import layers, models
import tensorflow as tf

# Define a function for creating the model
def build_model(hp):
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(128, (5, 5), activation='relu', input_shape=x_basic_train.shape[1:],
                      kernel_regularizer=regularizers.l2(0.01)),  # Add L2 regularization
        layers.BatchNormalization(),  # Normalize activations for better training
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),  # Dropout to prevent overfitting

        # Second Convolutional Block
        layers.Conv2D(64, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Third Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Fourth Convolutional Block
        layers.Conv2D(16, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Fully Connected Layers
        layers.Flatten(),
        layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Dropout(0.25),

        # Output Layer
        layers.Dense(num_classes, activation='softmax')  # Adjusted to match 36 classes
    ])

    # Compile the model with tunable learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# Baseline Model (Trained on Non-Augmented Data)

In [18]:
# Initialize the tuner
tuner_baseline = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3
)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Perform the search
tuner_baseline.search(x_basic_train, y_basic_train, epochs=10, validation_split=0.2, batch_size=32)



Trial 3 Complete [00h 00m 29s]
val_accuracy: 0.06400000303983688

Best val_accuracy So Far: 0.9399999976158142
Total elapsed time: 00h 01m 58s


In [20]:
# Retrieve the best model
best_hps_baseline = tuner_baseline.get_best_hyperparameters(num_trials=1)[0]

# Display the optimal hyperparameters
print(f"Best Hyperparameters:\n"
      f"Learning Rate: {best_hps_baseline.get('learning_rate')}")


Best Hyperparameters:
Learning Rate: 0.001


In [21]:

# Train the best model with the best hyperparameters
best_model_baseline = tuner_baseline.hypermodel.build(best_hps_baseline)
history_best_baseline = best_model_baseline.fit(
    x_basic_train,
    y_basic_train,
    epochs=50,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stopping])

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 71ms/step - accuracy: 0.2152 - loss: 4.8997 - val_accuracy: 0.8160 - val_loss: 3.1327
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.7290 - loss: 3.0126 - val_accuracy: 0.8520 - val_loss: 2.5298
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 60ms/step - accuracy: 0.8959 - loss: 1.9147 - val_accuracy: 0.8520 - val_loss: 2.0909
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.9643 - loss: 1.2413 - val_accuracy: 0.8320 - val_loss: 1.7961
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.9754 - loss: 0.8537 - val_accuracy: 0.8990 - val_loss: 1.4051
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - accuracy: 0.9834 - loss: 0.6101 - val_accuracy: 0.8650 - val_loss: 1.3734
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.9859 - loss: 0.4488 - val_accuracy: 0.9860 - val_loss: 0.6948
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.9892 - loss: 0.3491 - val

# Actual Model (Trained on Augmented Data)

In [22]:
# Initialize the tuner
tuner_model = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3
)

Reloading Tuner from ./untitled_project/tuner0.json


In [23]:
# Perform the search
tuner_model.search(x_augmented_train, y_augmented_train, epochs=20, validation_split=0.2, batch_size=32)



In [24]:
# Retrieve the best model
best_hps_model = tuner_model.get_best_hyperparameters(num_trials=1)[0]

# Display the optimal hyperparameters
print(f"Best Hyperparameters:\n"
      f"Learning Rate: {best_hps_model.get('learning_rate')}")


Best Hyperparameters:
Learning Rate: 0.001


In [25]:
# Train the best model with the best hyperparameters
early_stopping2 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
best_model = tuner_model.hypermodel.build(best_hps_baseline)
history_best = best_model.fit(
    x_augmented_train,
    y_augmented_train,
    epochs=50,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stopping2])

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 72ms/step - accuracy: 0.0607 - loss: 5.5117 - val_accuracy: 0.2900 - val_loss: 4.5699
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.2470 - loss: 4.3063 - val_accuracy: 0.5730 - val_loss: 3.6225
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.4680 - loss: 3.2938 - val_accuracy: 0.6410 - val_loss: 2.9450
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.6261 - loss: 2.5408 - val_accuracy: 0.7410 - val_loss: 2.2212
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.7209 - loss: 1.9768 - val_accuracy: 0.7270 - val_loss: 2.1023
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.7819 - loss: 1.6067 - val_accuracy: 0.8230 - val_loss: 1.6523
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.8087 - loss: 1.3656 - val_accuracy: 0.8240 - val_loss: 1.4603
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.8332 - loss: 1.1699 - val_ac

# Test



In [26]:
#from datasets.tests import affine
#from datasets.tests import center_rotation
#from datasets.tests import noise


In [27]:
# affine_results = []

# for dataset in affine.datasets:
#   all_affine_images, all_affine_labels = [], []
#   for i in range(50):
#     x, y = dataset[i]
#     all_affine_images.append(x)
#     all_affine_labels.append(y)

#   # Combine all batches into a single dataset
#   x_affine_test = tf.concat(all_affine_images, axis=0)
#   y_affine_test = np.concatenate(all_affine_labels, axis=0)

#   affine_results.append(best_model.evaluate(x_affine_test, y_affine_test))

# print(affine_results)

# Save Model

In [28]:
best_model.save('cnn_D.keras')
best_model_baseline.save('cnn_D_baseline.keras')